# Урок 9. Интеграция. Итоговый проект

Программа урока:


- Вспоминаем “типовой” процесс построения модели машинного обучения 
- Этапы “обучение” и “предсказание”
- Введение в json и rest

#### 1. "Типовой" процесс построения модели

<img src='https://drive.google.com/uc?export=view&id=1cHdANVRHhmqdPwkeWRHHMnqmwmyckeY4'>

Также нужно понимать разницу между этапом обучения и предсказания

<img src='https://drive.google.com/uc?export=view&id=1DUPlPmLwhbxW7j9ZEVuQdColxeUd1NnP' width=700>

Если говорить проще, то обучение модели - маленькая (пусть и важная) часть процесса решения конкретной бизнес-задачи

Сама по себе обученная модель бесполезна, если нет возможности с ней взаимодействовать

#### Пример

Интернет-магазин "ходит" к нам за рекомендациями для пользователя.

Т.е нам передают user_id и просят вернуть топ k рекомендованных товаров.

В общем виде процесс "выкатывания" модели в прод может выглядеть так:

<img src='https://drive.google.com/uc?export=view&id=1J0ZW0O_5sl-WOE4FWfQT_CNTBmQWFUK7'>

Этап Model Deployment может быть очень сложным и нетривиальным во всем процессе (по сравнению с этапом обучения модели и подготовки данных).

#### Пример реализации проекта

<img src='https://drive.google.com/uc?export=view&id=1ymysdP0wVthPV6q8lZ6LKbaRh-6dHkjO' width=600>

https://hackmd.io/@AndreyPhys/flask-ml-api#/

### REST API

https://ru.wikipedia.org/wiki/REST

<img src='https://drive.google.com/uc?export=view&id=1waf6XV_4n6chq2gFnqUHErs87MfKqqQT' width=700>

### JSON

<img src='https://drive.google.com/uc?export=view&id=1okkgmx5vU1BQlWyW5k0djYkkF_czhn0k' width=700>

### Flask

- установить
- создать объект Flask
- написать обработчики (handler)
- запустить (и не выключать)

Создать объект Flask:



```
from flask import Flask, request, jsonify

app = Flask(__name__)  # __name__ - имя модуля
```


Написать обработчики (handler):



```
@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"
```


Запустить (и не выключать):


```
if __name__ == "__main__":
    app.run()
```




# Step 1 - TRAIN

### Обучение пайплайна

1. Загрузим данные https://www.kaggle.com/shivamb/real-or-fake-fake-jobposting-prediction
2. Соберем пайплайн с простейшим препроцессингом (tfidf) на текстовых данных
3. Обучим логистическую регрессию и сохраним на диск предобученный пайплайн

In [ ]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

Загрузим данные

Ссылка на google drive: https://drive.google.com/file/d/1T0V44fGNCbgaWh78gZ2ieeoqo5eBowVi

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1T0V44fGNCbgaWh78gZ2ieeoqo5eBowVi' -O fake_job_postings.csv

--2022-06-10 17:13:55--  https://drive.google.com/uc?export=download&id=1T0V44fGNCbgaWh78gZ2ieeoqo5eBowVi
Resolving drive.google.com (drive.google.com)... 209.85.147.138, 209.85.147.102, 209.85.147.139, ...
Connecting to drive.google.com (drive.google.com)|209.85.147.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/rust99jtvmfi2397cnku10jhpkok8mul/1654881225000/14904333240138417226/*/1T0V44fGNCbgaWh78gZ2ieeoqo5eBowVi?e=download [following]
--2022-06-10 17:13:59--  https://doc-10-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/rust99jtvmfi2397cnku10jhpkok8mul/1654881225000/14904333240138417226/*/1T0V44fGNCbgaWh78gZ2ieeoqo5eBowVi?e=download
Resolving doc-10-c0-docs.googleusercontent.com (doc-10-c0-docs.googleusercontent.com)... 142.250.125.132, 2607:f8b0:4001:c2f::84
Connecting to doc-10-c0-docs.googleusercontent.com (doc-10-c0-doc

In [ ]:
df = pd.read_csv("./fake_job_postings.csv")
df.head(3)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
df['fraudulent'].value_counts()

0    17014
1      866
Name: fraudulent, dtype: int64

Разделим данные на train/test и сохраним тестовую выборку на диск

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, df['fraudulent'],
                                                    test_size=0.33, random_state=42)
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    

class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [ ]:
features = ['description', 'company_profile', 'benefits']
target = 'fraudulent'

Соберем кусок, ответственный за feature engineering

In [ ]:
# combine
description = Pipeline([
                ('imputer', TextImputer('description', '')),
                ('selector', ColumnSelector(key='description')),
                ('tfidf', TfidfVectorizer())
            ])

company_profile = Pipeline([
                ('imputer', TextImputer('company_profile', '')),
                ('selector', ColumnSelector(key='company_profile')),
                ('tfidf', TfidfVectorizer())
            ])

benefits = Pipeline([
                ('imputer', TextImputer('benefits', '')),
                ('selector', ColumnSelector(key='benefits')),
                ('tfidf', TfidfVectorizer())
            ])


feats = FeatureUnion([('description', description),
                      ('company_profile', company_profile),
                      ('benefits', benefits)])

Добавим простейший классификатор

In [ ]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

CPU times: user 5.55 s, sys: 2.39 s, total: 7.93 s
Wall time: 5.89 s


In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    

class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X
    
# combine
description = Pipeline([
                ('imputer', TextImputer('description', '')),
                ('selector', ColumnSelector(key='description')),
                ('tfidf', TfidfVectorizer())
            ])

company_profile = Pipeline([
                ('imputer', TextImputer('company_profile', '')),
                ('selector', ColumnSelector(key='company_profile')),
                ('tfidf', TfidfVectorizer())
            ])

benefits = Pipeline([
                ('imputer', TextImputer('benefits', '')),
                ('selector', ColumnSelector(key='benefits')),
                ('tfidf', TfidfVectorizer())
            ])


feats = FeatureUnion([('description', description),
                      ('company_profile', company_profile),
                      ('benefits', benefits)])

%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

Посмотрим, как выглядит наш pipeline

In [ ]:
pipeline.steps

[('features', FeatureUnion(transformer_list=[('description',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='description',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='description')),
                                                  ('tfidf', TfidfVectorizer())])),
                                 ('company_profile',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='company_profile',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='company_profile')),
                                                  ('tfidf

Сохраним модель (пайплайн)

In [ ]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

# Step 2 - PREDICT

### Проверка работоспособности и качества пайплайна

Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [ ]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [ ]:
X_test.head(3)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,4709,Python Engineer,"GB, , London",NaN,NaN,NaN,Stylect is a dynamic startup that helps helps ...,We don’t care where you studied or what your G...,We are negotiable on salary and there is the p...,0,1,0,Full-time,Entry level,Unspecified,Apparel & Fashion,Information Technology,0
1,11080,Entry Level Sales,"US, OH, Cincinnati",NaN,55000-75000,NaN,General Summary: Achieves maximum sales profit...,NaN,Great Health and DentalFast Advancement Opport...,1,0,0,Full-time,Entry level,High School or equivalent,Financial Services,Sales,0
2,12358,Agile Project Manager,"US, NY, New York",NaN,NaN,ustwo offers you the opportunity to be yoursel...,"At ustwo™ you get to be yourself, whilst deliv...",Skills• Experience interfacing directly with c...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [ ]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('description',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='description',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='description')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('company_profile',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='company_profile',
                                                     

In [ ]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [ ]:
preds[:10]

array([0.02602203, 0.04317015, 0.00370601, 0.00112958, 0.00151454,
       0.00213981, 0.00256837, 0.00373913, 0.00069803, 0.0122069 ])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.20397799406875214, F-Score=0.852, Precision=0.900, Recall=0.809


# Step 3 - FLASK

## При внедрении

**При внедрении необходимо сделать:**
*   Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
*   Определить ip-адрес и порт, на который будут поступать данные.
*   Создать во Flask необходимые роуты:<br/>
    `@app.route('/predict_example', method='POST')`<br/>
    `def predict_example():`
*   Перенести во Flask все функции преобразования данных,
    *   формат данные, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
*   Загрузить обученные модели.
*   Настроить логирование.

## Flask

Тут будет сервис для обработки запросов на Flask

Google Colab предоставляет виртуальную машину, поэтому мы не можем получить доступ к локальному хосту, как это делаем на нашем локальном компьютере при запуске локального веб-сервера. Что мы можем сделать, так это предоставить его общедоступному URL-адресу с помощью ngrok.

https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b

In [ ]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

https://dashboard.ngrok.com/get-started/setup

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 25vEpcJ5Ih4vlUp4thEZ9sEA6ZU_3Bnu17gKacRXhF6hLeefc
!./ngrok http 80

--2022-06-10 17:24:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  38.1MB/s    in 0.3s    

2022-06-10 17:24:07 (38.1 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Пробный запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d65a-35-202-7-199.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Jun/2022 17:24:36] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2022 17:24:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2022 17:24:59] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2022 17:25:06] "GET /a HTTP/1.1" 200 -


In [ ]:
import pandas as pd
import dill

### **Создаем сервис для обработки запросов к модели**

In [ ]:
# Загружаем обученные модели
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [ ]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Запустить сервис и не глушить его, пока работаем 

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    description, company_profile, benefits = "", "", ""
    request_json = request.get_json()
    
    if request_json["description"]:
        description = request_json['description']
    
    if request_json["company_profile"]:
        company_profile = request_json['company_profile']
                
    if request_json["benefits"]:
        benefits = request_json['benefits']
    
    print(description)  
    preds = model.predict_proba(pd.DataFrame({'Home Ownership' : [home_ownership],
                                            'Annual Income' : [annual_income],
                                            'Years in current job' : years_job,
                                            'Tax Liens' : tax_liens,
                                            'Number of Open Accounts' : accounts,
                                            'Years of Credit History' : years_history,
                                            'Maximum Open Credit' : max_credit,
                                            'Number of Credit Problems' : n_problems,
                                            'Months since last delinquent' : last_delinquent,
                                            'Bankruptcies' : bankruptcies,
                                            'Purpose' : purpose,
                                            'Term' : term,
                                            'Current Loan Amount' : current_loan_amount ,
                                            'Current Credit Balance' : current_credit_balance,
                                            'Monthly Debt' : monthly_debt,
                                            'Credit Score' : credit_score}))
    data["predictions"] = preds[:, 1][0]
    data["description"] = description
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://afcc-35-202-7-199.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Jun/2022 17:36:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2022 17:36:33] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2022 17:39:00] "POST /predict HTTP/1.1" 200 -


Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them. As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity.  Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.
OK


...

To apply please visit our website at #URL_06ae9636e61d7ddfc75b7dec9887f7022036b464a1ef22d098f1e03084cd3614# and click on our Careers page.Tidewater Finance Company is seeking full-time RECOVERY SPECIALISTS. Join a growing team of high performance professionals in a team-oriented environment! The qualified applicant must be able to:Properly and independently work assigned accounts to locate customer and/or collateral by performing advanced loss prevention activities Perform basic and advanced skip-tracing with the use of internal and external skip-tracing resourcesNegotiate account resolution and accurately input and document all actions within the collections systemMonitor and measure performance of third party repo agents and other outside vendors to ensure goals are achieved in the most cost effective mannerEnsure all company policies and procedures are adhered toAlert management of potential risk exposure The qualifications for this position include:A professional demeanorAdaptabili

127.0.0.1 - - [10/Jun/2022 17:40:57] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2022 17:40:57] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2022 17:40:57] "POST /predict HTTP/1.1" 200 -


Please apply for the position as a .Net Developer at In2media by clicking the "Apply for this job"-button below.We are looking forward to receiving your application.In2media
OK
Kappa Search Inc. is a Chicago technical staffing company that specializes in engineering, manufacturing, technical sales and supply chain recruitment and placement. We are currently recruiting for a mechanical design engineer with product development experience. Responsibilities and requirements as follows.Responsibilities:Responsible for producing 3D CAD and other technical drawings for all new product designs and existing products.To provide the required design services (such as CAD &amp; CAE) for NPD and support of existing products and their application, as well as for the modification and further development of these same products.Prepares design concepts, layouts, assembly drawings and schematics as part of the development team.Material and component selection; develops specs for product.Pepares cost esti

127.0.0.1 - - [10/Jun/2022 17:40:57] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2022 17:40:57] "POST /predict HTTP/1.1" 200 -


At Intent HQ we’re tackling some seriously difficult problems, right at the cutting edge of deep consumer analysis. We model user interests and apply this insight to solve challenging consumer problems at scale. Want to draw insights from 20 million detailed social network profiles? In realtime? We do.To help us innovate faster, we’re building a new R&amp;D group. This team is responsible for researching, designing and prototyping algorithms in the machine learning and NLP space. We have an engineering team responsible for the overall platform, who you will work closely with to bring prototypes to production.
OK
Our client is searching for a strong candidate that has 2 years Grails development experience as well as 3 years Java development experience.This is an excellent opportunity for an ambitious, creative developer that wants to innovate and build what doesn’t exist yet. Working remotely for a well funded company with an existing distributed team. Our client is a cloud platform pro

Сторона клиента: https://colab.research.google.com/drive/1UK_ToiHKZaZhKt8nZmAlWDIb0Nhz_c5X

Тестовый клиент

In [ ]:
# Пример данных
description_data, company_profile_data, benefits_data = ( 
    "Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them.\xa0As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity. \xa0Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.",
    "ustwo offers you the opportunity to be yourself, whilst delivering the best work on the planet for some of the biggest and most innovative brands. A culture thriving on collaboration underpins what is an amazing work smart/ live well environment.We genuinely care about the work that we deliver and the people who help make it all possible. We only invest in projects, people and practices that we believe in, to ensure we remain excited about every opportunity.",
    "We are negotiable on salary and there is the potential for equity for the right candidate."
)

body = {
        'description': description_data, 
        'company_profile': company_profile_data,
        'benefits': benefits_data
        }

In [ ]:
with app.test_client() as t:
    response = t.post('/predict', json=body)
    json_data = response.get_json()

json_data

Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them. As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity.  Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.
OK


{'description': 'Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them.\xa0As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity. \xa0Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.',
 'predictions': 0.001129359819418078,
 'success': Tr

# Домашнее задание

**Стандартная версия**
Нужно реализовать rest api на базе flask в google colab.

1. выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
2. Реализовать ноутбук с сервером
3. Реализовать ноутбук с клиентом


**Сложная версия**

Нужно реализовать rest api на базе flask (пример https://github.com/fimochka-sudo/GB_docker_flask_example)

1. выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
2. для вашего проекта вам понадобится requirements.txt с пакетами. Можно за основу взять такой файл из проекта выше. Для его установки прям в pycharm можно открыть терминал и сделать pip install -r requirements.txt (находясь в корне проекта)
3. итоговый проект должен содержать:
    1. каталог app/models/ (здесь модель-пайплайн предобученная либо код обучения модели-пайплайна)
    2. файл app/run_server.py (здесь основной код flask-приложения)
    3. requirements.txt (список пакетов, которые у вас используются в проекте - в корне проекта)
    4. README.md (здесь какое-то описание, что вы делаете, что за данные, как запускать и т.д)
    5. *Dockerfile
    6. *docker-entrypoint.sh
4. *front-end сервис какой-то, который умеет принимать от пользователя введеные данные и ходить в ваш api. На самом деле полезно больше вам, т.к если ваш проект будет далее развиваться (новые модели, интересные подходы), то это хороший пунктик к резюме и в принципе - строчка в портфолио)

## Полезные ссылки:

1. датасеты (для полета мысли): https://www.kaggle.com/datasets
2. конкурс Сбербанка по недвижимости (можно этот набор данных также взять и обучить модель предсказывать стоимость жилья - неплохой такой сервис может получиться) - https://www.kaggle.com/c/sberbank-russian-housing-market/data Там же и ноутбуки с разными подходами есть.
3. минималистичный пример связки keras/flask https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html для определения класса картинки
4. неплохой такой пример связки docker/flask - https://cloud.croc.ru/blog/byt-v-teme/flask-prilozheniya-v-docker/
5. https://www.digitalocean.com/community/tutorials/how-to-build-and-deploy-a-flask-application-using-docker-on-ubuntu-18-04
6. https://flask.palletsprojects.com/en/2.1.x/


In [4]:
import numpy as np

In [11]:
a, b = [1] *2

In [12]:
type(b)

int